In [1]:
import tensorflow as tf

# Enable memory growth to allocate only as needed
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

# Alternatively, set a memory limit if you want to control the allocation
# Uncomment the lines below and adjust the limit as needed
memory_limit = 4096  # Set memory limit in MB (e.g., 4GB)
tf.config.set_logical_device_configuration(
    gpu,
    [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
)

2024-10-12 06:13:57.163513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 06:13:57.178583: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 06:13:57.183174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 06:13:57.195229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1728713640.264469    1107 cuda_executor.c

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Updated Configuration
CONFIGURATION = {
    'BATCH_SIZE': 32,
    'IM_SIZE': 224,
    'N_EPOCHS': 10,
    'LEARNING_RATE': 1e-3,
    'NUM_CLASSES': 3,
}

# Load datasets
train_dataset = image_dataset_from_directory(
    '../../EmotionsDataset/train/',
    image_size=(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE']),
    batch_size=CONFIGURATION['BATCH_SIZE'],
    label_mode='int'
)

test_dataset = image_dataset_from_directory(
    '../../EmotionsDataset/test/',
    image_size=(CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE']),
    batch_size=CONFIGURATION['BATCH_SIZE'],
    label_mode='int'
)

# Build the model
def build_model(config):
    base_model = EfficientNetB0(include_top=False, input_shape=(config['IM_SIZE'], config['IM_SIZE'], 3), weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(config['NUM_CLASSES'], activation='softmax')
    ])

    return model

model = build_model(CONFIGURATION)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Found 6799 files belonging to 3 classes.


I0000 00:00:1728713646.245332    1107 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728713646.245429    1107 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728713646.245456    1107 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728713646.352004    1107 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-12 06:14:06.352042: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2112] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.

Found 2280 files belonging to 3 classes.


In [3]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=CONFIGURATION['N_EPOCHS']
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/10


I0000 00:00:1728713666.019834    1188 service.cc:146] XLA service 0xe8aadc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728713666.019923    1188 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-10-12 06:14:26.345215: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-12 06:14:27.597482: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-10-12 06:14:30.012597: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11307', 148 bytes spill stores, 148 bytes spill loads

2024-10-12 06:14:30.518323: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory 

212/213 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4866 - loss: 1.4309

2024-10-12 06:15:00.802082: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11307', 8 bytes spill stores, 8 bytes spill loads



213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.4869 - loss: 1.4298

2024-10-12 06:15:27.649861: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2074', 8 bytes spill stores, 8 bytes spill loads



213/213 ━━━━━━━━━━━━━━━━━━━━ 86s 236ms/step - accuracy: 0.4872 - loss: 1.4287 - val_accuracy: 0.6531 - val_loss: 0.7695
Epoch 2/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.6210 - loss: 0.9166 - val_accuracy: 0.7373 - val_loss: 0.6371
Epoch 3/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.6825 - loss: 0.7630 - val_accuracy: 0.7500 - val_loss: 0.6029
Epoch 4/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7021 - loss: 0.6942 - val_accuracy: 0.7592 - val_loss: 0.5783
Epoch 5/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7238 - loss: 0.6470 - val_accuracy: 0.7724 - val_loss: 0.5622
Epoch 6/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7313 - loss: 0.6236 - val_accuracy: 0.7772 - val_loss: 0.5610
Epoch 7/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7393 - loss: 0.6132 - val_accuracy: 0.7706 - val_loss: 0.5635
Epoch 8/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - accuracy: 0.7422 - loss: 0.5952 - val_accura

In [17]:
def representative_data_gen():
    for input_value, _ in train_dataset.take(1):
        yield [input_value]
representative_data_gen()

<generator object representative_data_gen at 0x7f3e662468e0>

In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
def representative_data_gen():
    for input_value, _ in train_dataset.take(100):
        yield [input_value]
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpvxe46hw2/assets


INFO:tensorflow:Assets written to: /tmp/tmpvxe46hw2/assets


Saved artifact at '/tmp/tmpvxe46hw2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_238')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  139903579454160: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139903579455120: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139903579454736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903459148432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903459148240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903579455312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903459149008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903459150160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903459150544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139903459148048: TensorSpec(shape=(), dtype=tf.resource, name=

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1728724488.727876    1107 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1728724488.728076    1107 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-12 09:14:48.728775: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvxe46hw2
2024-10-12 09:14:48.755583: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-12 09:14:48.755639: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpvxe46hw2
2024-10-12 09:14:49.092015: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-12 09:14:50.954270: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpvxe46hw2
2024

In [7]:
!python -m tf2onnx.convert --saved-model effnetmodel --output effnetmodel.onnx

<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2024-10-12 08:58:15,252 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
I0000 00:00:1728723496.370022    1761 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728723496.474272    1761 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728723496.474472    1761 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728723496

In [4]:
effnet = 'effnetmodel.keras'
model.save(effnet)

In [6]:
import tensorflow as tf
import tf2onnx

# Load your Keras model (.h5 format)
effnet = 'effnetmodel.keras'
model = tf.keras.models.load_model(effnet)


ValueError: Layer "dense" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1258>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1259>]